In [ ]:
import os
import sys
from pathlib import Path

sys.path.append("../")

import numpy as np
import pandas as pd

In [ ]:
from choice_learn.datasets.examples import load_tafeng

In [ ]:
df = load_tafeng(as_frame=True)
df.head()

In [ ]:
dataset = load_tafeng(as_frame=False, preprocessing="assort_example")

In [ ]:
dataset.contexts_items_features[0].shape

In [ ]:
dataset.summary()